In [34]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
from itertools import combinations
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [35]:
data = pd.read_csv('data/assignment_data.csv', sep=';')

In [36]:
data.shape

(2572, 55)

In [3]:
ids = ['country_id','application_id','product_id','customer_id']
dates = ['due_date','first_status_day_date','first_status_time_of_day','paid_date','arrived_date','Variable_42','Variable_43','Variable_44']
categoricals = ['Variable_5','Variable_6','Variable_12','Variable_45']
ordinals = ['Variable_13','Variable_14']
numericals = [x for x in data.columns if (x not in ids) and (x not in dates) and (x not in categoricals) and (x not in ordinals) and x != 'Target']

# Categorical and Ordinals

In [4]:
def transform_categorical(dataset, columns_to_transform):
    """Translates categorical columns into integers."""

    categorical_dictionary = {}
    for column in columns_to_transform:
        categories = pd.Categorical(dataset[column])
        new_column = '{prefix}{suffix}'.format(prefix=column, suffix='_INT')
        dataset.loc[:, new_column] = categories.codes
        categorical_dictionary[column] = dict([(k, v) for v, k in enumerate(categories.categories)])
        
    return dataset, categorical_dictionary

In [5]:
data, categorical_dictionary = transform_categorical(data, categoricals)
data, ordinal_dictionary =  transform_categorical(data, ordinals)

In [6]:
data.shape

(2572, 61)

# Dates

In [7]:
for date in dates:
    data[date] = pd.to_datetime(data[date])

In [8]:
def transform_dates_day(dataset, columns_to_transform):
    """Translates dates columns into integers. Granularity: Year, Month, Day"""
    
    for column in columns_to_transform:
        new_column_year = '{prefix}{suffix}'.format(prefix=column, suffix='_YEAR')
        new_column_month = '{prefix}{suffix}'.format(prefix=column, suffix='_MONTH')
        new_column_day = '{prefix}{suffix}'.format(prefix=column, suffix='_DAY')
        
        dataset.loc[:, new_column_year] = dataset.loc[:, column].dt.year
        dataset.loc[:, new_column_month] = dataset.loc[:, column].dt.month
        dataset.loc[:, new_column_day] = dataset.loc[:, column].dt.day
        
    return dataset

In [9]:
def transform_dates_second(dataset, columns_to_transform):
    """Translates dates columns into integers. Granularity: Hour, Minute, Second"""
    
    for column in columns_to_transform:
        new_column_hour = '{prefix}{suffix}'.format(prefix=column, suffix='_HOUR')
        new_column_minute = '{prefix}{suffix}'.format(prefix=column, suffix='_MINUTE')
        new_column_second = '{prefix}{suffix}'.format(prefix=column, suffix='_SECOND')
        
        dataset.loc[:, new_column_hour] = dataset.loc[:, column].dt.hour
        dataset.loc[:, new_column_minute] = dataset.loc[:, column].dt.minute
        dataset.loc[:, new_column_second] = dataset.loc[:, column].dt.second
        
    return dataset

In [10]:
data = transform_dates_day(data, ['due_date','first_status_day_date','paid_date','arrived_date','Variable_42','Variable_43','Variable_44'])
data = transform_dates_second(data, ['first_status_time_of_day','arrived_date'])

In [11]:
data.shape

(2572, 88)

### Feature Engineering: Adding distance between dates 

In [12]:
for couple in combinations(['due_date','first_status_day_date','paid_date','arrived_date','Variable_42','Variable_43','Variable_44'],2):
    columnn_name = "DAYS_{}-{}".format(couple[0],couple[1])
    days = (data[couple[0]] - data[couple[1]]).dt.days
    data[columnn_name] = days

In [13]:
data.shape

(2572, 109)

# Drop Transformed Variables

In [14]:
data = data.drop(categoricals, axis=1)
data = data.drop(ordinals, axis=1)
data = data.drop(dates, axis=1)

# Drop Constant Columns

In [15]:
def drop_constant_column(dataframe):
    """Drops constant value columns of pandas dataframe."""
    
    return dataframe.loc[:, (dataframe != dataframe.iloc[0]).any()]

In [16]:
data = drop_constant_column(data)

In [17]:
data.shape

(2572, 90)

In [18]:
for c in data.columns:
    if c in ids:
        print(c)

application_id
customer_id


# Drop Empty Columns  

# Inputing NaN

In [32]:
total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Emptyness Percentage'])
missing_data.head(20)

,Total,Emptyness Percentage
DAYS_Variable_43-Variable_44,0,0.0
due_date_MONTH,0,0.0
Variable_28,0,0.0
Variable_29,0,0.0
Variable_30,0,0.0
Variable_31,0,0.0
Variable_32,0,0.0
Variable_33,0,0.0
Variable_34,0,0.0
Variable_35,0,0.0


In [20]:
for column in [x for x in data.columns if x != 'Target']:
    #data[column] = data[column].fillna((data[column].mean()))
    data[column] = data[column].fillna(0)

# Grab Ferratum Test set

In [21]:
ferratum_preprocessed_test_set = data[data.Target.isnull()]

In [22]:
ferratum_preprocessed_test_set.shape

(515, 90)

In [23]:
ferratum_preprocessed_test_set.to_csv('data/ferratum_preprocessed_test_set.csv', sep=';', index=False)

# Split in Train Test

In [24]:
data = data[data.Target.notnull()]

In [25]:
target = data['Target']

In [26]:
data.drop(['Target'], 1 ,inplace=True)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(
    data, 
    target, 
    test_size=0.25, 
    random_state=2019,
    stratify = target)

In [28]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1542, 89)
(1542,)
(515, 89)
(515,)


In [29]:
data.to_csv('data/preprocessed_entire_set.csv', sep=';', index=False)
target.to_csv('data/entire_target.csv', sep=';', index=False, header='Target')

X_train.to_csv('data/preprocessed_train_set.csv', sep=';', index=False)
y_train.to_csv('data/train_target.csv', sep=';', index=False, header='Target')

X_test.to_csv('data/preprocessed_test_set.csv', sep=';', index=False)
y_test.to_csv('data/test_target.csv', sep=';', index=False,  header='Target')